In [1]:
import sagemaker
import torch
import torchvision
import os
from PIL import Image

In [2]:
print(torch.__version__)
print(torchvision.__version__)

1.4.0
0.5.0


In [3]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

In [4]:
model_path = 'model/'
code_path = 'code/'

In [35]:
prefix = 'pytorch_dogscats'
model_path = sess.upload_data(path='dogscats.tar.gz', bucket=bucket, key_prefix=prefix)
print(model_path)

s3://sagemaker-us-east-2-805291263703/pytorch_dogscats/dogscats.tar.gz


In [5]:
from sagemaker.pytorch.model import PyTorchPredictor, PyTorchModel

## Using mxnet-model-server

In [29]:
model = PyTorchModel(model_data=model_path, 
                     role=role, 
                     entry_point='inference.py', 
                     framework_version='1.5', 
                     py_version='py3')

In [16]:
endpoint_name = 'pytorch-mxnet'

In [36]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.large', endpoint_name=endpoint_name)

--------------!

## Using torchserve

In [6]:
from sagemaker.utils import name_from_base

In [ ]:
model = PyTorchModel(model_data=model_path,
                     name=name_from_base('pytorch-torchserve'),
                     role=role, 
                     entry_point='inference.py', 
                     framework_version='1.8.0',
                     py_version='py3', 
                    )

In [ ]:
predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.m5.large', 
                         endpoint_name=name_from_base('pytorch-torchserve'))

# Predict

In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [10]:
class_demap = {
    1: 'Dog',
    0: 'Cat',
}

In [11]:
test_imgs = os.listdir('./test1')

In [52]:
# content_type = 'application/x-npy' # Default
content_type = 'application/json'

In [53]:
if content_type == 'application/x-npy':
    predictor.serializer = sagemaker.serializers.NumpySerializer()
elif content_type == 'application/json':
    predictor.serializer = sagemaker.serializers.JSONSerializer()

In [55]:
# for i in range(len(test_imgs)):
for i in range(1):
    image_name = test_imgs[i]
    im = cv2.imread(os.path.join('./test1', image_name))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (224,224))
    tensor_x = np.expand_dims(im, axis=0)
    tensor_x = tensor_x.transpose((0,3,1,2))
    
    if content_type == 'application/x-npy':
        payload = torch.from_numpy(tensor_x).float()
    elif content_type == 'application/json':
        payload = {
            'url': tensor_x
        }
#     tensor_x = torch.from_numpy(tensor_x).float()
    response = predictor.predict(payload)[0][0]
    print(f'Image:{image_name}, predicted: {class_demap[response > 0.5]}')
    print(f'Probability: {response}')
    plt.imshow(im)
    plt.show()

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint pytorch-mxnet of account 805291263703 not found.

# Delete endpoint

In [51]:
predictor.delete_endpoint()